## Third kind of the task of querying : What is the most related XXX to YYY?
We can use BRICK module to query the related nodes of YYY in the type of XXX.Then we can rank them, finally get the most related node.

In [1]:
user_question = "Which is disease is most related to the gene Isl1?"

Load the BRICK module.

In [ ]:
import sys
import os
import BRICK
import scanpy as sc

from dotenv import dotenv_values
import json
config = dotenv_values('/workspace/data/brick.env')
kg_url = config.get('KG_URL')
kg_auth = (config.get('KG_AUTH_USER'), config.get('KG_AUTH_PASS'))
model = config.get("MODEL_TYPE")
url = config.get("LLM_URL")
api_key = config.get("API_KEY")
llm_params = json.loads(config.get("LLM_PARAMS"))

BRICK.config(url=kg_url, auth=kg_auth)
BRICK.config_llm(modeltype=model, base_url=url,api_key=api_key, llm_params=llm_params)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Graph database has been configured and initialized successfully.
LLM has been configured and initialized successfully.


Then use BRICK module to query the related disease of gene Isl1.

In [3]:
source_entity = BRICK.inp.extract_entities_neighbor(user_question)
print(source_entity)

(['Isl1'], 'Gene', 'Disease')


In [4]:
client = BRICK.se.BRICKSearchClient()
print(f"字符串索引: {client.search_config.string_index_name}")
print(f"向量索引: {client.search_config.vector_index_name or '已禁用'}\n")

payload_fuzzy_auto = client.build_untyped_payload(query_text=source_entity, top_k=1)
print(f">>> [Fuzzy] 未指定类别：全类别检索（query_text='{source_entity}'）")
df_all = client.search_fuzzy(payload_fuzzy_auto)
entity_id = df_all.head(1)['entity_id'].iloc[0]
print(entity_id)


字符串索引: brick_index_*_string
向量索引: brick_index_*_vector

>>> [Fuzzy] 未指定类别：全类别检索（query_text='(['Isl1'], 'Gene', 'Disease')'）
NCBI:16392


In [5]:
entity_name = df_all.head(1)['primary_name'].iloc[0]

In [6]:
most_rel_disease = BRICK.qr.query_neighbor(source_entity_set=entity_name, source_entity_type="Gene",target_entity_type="Disease",return_type="dataframe")
# most_rel_disease


Use rk.match_count to get the most related diseases.

In [7]:
res = BRICK.rk.info_source_count(most_rel_disease)


Use the BRICK.inp.interpret_query to get the final answer.

In [8]:
ans = BRICK.inp.interpret_query("What are the most related diseases to Isl1?",res)
print(ans)

Based on the provided table, the most related diseases to Isl1, considering the frequency and the type of relations, are as follows:

- **Type 2 Diabetes Mellitus (DOID:9352)**: This disease is listed with a "marker_of" relation to Isl1, which indicates that Isl1 might be used as a marker for this condition. The confidence score for this relation is 0.5, and it has the highest info_source_count of 3, suggesting a stronger or more frequently observed association.
- **Congenital Heart Disease (DOID:1682)**: This disease is sequentially related to Isl1, indicating a stepwise or developmental relationship. It has an info_source_count of 1.
- **Myocardial Infarction (DOID:5844)**: Also sequentially related to Isl1, with an info_source_count of 1.
- **Type 1 Diabetes Mellitus (DOID:9744)**: Similarly, this disease is sequentially related to Isl1, with an info_source_count of 1.
- **Ventricular Septal Defect (DOID:1657)**: This disease is also sequentially related to Isl1, with an info_source